In [1]:
import pandas as pd
import numpy as np

# Import Excel Files of the Past 2 NBA Seasons

In [2]:
team_2016_2017 = pd.read_excel('2016-2017_NBA_Box_Score_Team-Stats.xlsx')
team_2017_2018 = pd.read_excel('2017-2018_NBA_Box_Score_Team-Stats.xlsx')
player_2016_2017 = pd.read_pickle('player_2016_2017_w_hand_calc_DKPTS.pkl')
player_2017_2018 = pd.read_pickle('player_2017_2018_w_hand_calc_DKPTS.pkl')

DFS_2016_2017 = pd.read_excel('NBA-2016-2017-DFS-Dataset.xlsx')
DFS_2017_2018 = pd.read_excel('NBA-2017-2018-DFS-Dataset.xlsx')

In [3]:
#Rename the columns to remove spaces
player_2016_2017 = player_2016_2017.rename(index=str, columns={'DATA SET':'DATASET','PLAYER FULL NAME':'PLAYER','OWN TEAM':'TEAM','OPP TEAM':'OPPONENT','VENUE (R/H)':'ROAD_HOME','MIN':'MINUTES'})
player_2017_2018 = player_2017_2018.rename(index=str, columns={'DATA SET':'DATASET','PLAYER FULL NAME':'PLAYER','OWN TEAM':'TEAM','OPP TEAM':'OPPONENT','VENUE (R/H)':'ROAD_HOME','MIN':'MINUTES'})

In [4]:
#Select only regular season games
team_2016_2017 = team_2016_2017.loc[team_2016_2017['DATASET'] == '2016-2017 Regular Season']
team_2017_2018 = team_2017_2018.loc[team_2017_2018['DATASET'] == '2017-2018 Regular Season']
player_2016_2017 = player_2016_2017.loc[player_2016_2017['DATASET'] == '2016-2017 Regular Season']
player_2017_2018 = player_2017_2018.loc[player_2017_2018['DATASET'] == '2017-2018 Regular Season']
DFS_2016_2017 = DFS_2016_2017.loc[DFS_2016_2017['DATASET'] == '2016-2017 Regular Season']
DFS_2017_2018 = DFS_2017_2018.loc[DFS_2017_2018['DATASET'] == '2017-2018 Regular Season']

In [5]:
#This is a list of a unique teams
teams = team_2016_2017.TEAMS.unique()

# Create New Features

In [6]:
#column that is absolute value of spread
team_2016_2017['SPREAD'] = abs(team_2016_2017['OPENING SPREAD'])
team_2017_2018['SPREAD'] = abs(team_2017_2018['OPENING SPREAD'])

#Column singnifying if they are favorites
team_2016_2017['FAVORITE'] = team_2016_2017['MONEYLINE'] < 0
team_2017_2018['FAVORITE'] = team_2017_2018['MONEYLINE'] < 0

#Column signifying a spread greater than 10
team_2016_2017['SPREAD_10+'] = team_2016_2017['SPREAD'] >= 10
team_2017_2018['SPREAD_10+'] = team_2017_2018['SPREAD'] >= 10

In [7]:
#This code creates DFs for DvP stats for each game
df16_17 = player_2016_2017[['DATE','TEAM','POSITION','DraftKings_PTS']]
df17_18 = player_2017_2018[['DATE','TEAM','POSITION','DraftKings_PTS']]

positional_pts_16_17 = df16_17.groupby(['DATE','TEAM','POSITION']).sum()
positional_pts_17_18 = df17_18.groupby(['DATE','TEAM','POSITION']).sum()

positional_pts_16_17 = positional_pts_16_17.unstack(level=-1)
positional_pts_17_18 = positional_pts_17_18.unstack(level=-1)

positional_pts_16_17 = positional_pts_16_17.xs('DraftKings_PTS', axis=1, drop_level=True)
positional_pts_17_18 = positional_pts_17_18.xs('DraftKings_PTS', axis=1, drop_level=True)

positional_pts_16_17 = positional_pts_16_17.reset_index(level=1)
positional_pts_17_18 = positional_pts_17_18.reset_index(level=1)

positional_pts_16_17 = positional_pts_16_17.reset_index()
positional_pts_17_18 = positional_pts_17_18.reset_index()

positional_pts_16_17 = positional_pts_16_17.rename(columns={'TEAM':'TEAMS','C':'C_DK_PTS','F':'F_DK_PTS','G':'G_DK_PTS','PF':'PF_DK_PTS','PG':'PG_DK_PTS','SF':'SF_DK_PTS','SG':'SG_DK_PTS'})
positional_pts_17_18 = positional_pts_17_18.rename(columns={'TEAM':'TEAMS','C':'C_DK_PTS','F':'F_DK_PTS','G':'G_DK_PTS','PF':'PF_DK_PTS','PG':'PG_DK_PTS','SF':'SF_DK_PTS','SG':'SG_DK_PTS'})

positional_pts_16_17 = positional_pts_16_17.set_index(['DATE','TEAMS'])
positional_pts_17_18 = positional_pts_17_18.set_index(['DATE','TEAMS'])

In [8]:
team_2016_2017 = team_2016_2017.join(positional_pts_16_17,on=['DATE','TEAMS'])
team_2017_2018 = team_2017_2018.join(positional_pts_17_18,on=['DATE','TEAMS'])

In [9]:
team_2016_2017 = team_2016_2017.rename(columns={'TEAMS':'TEAM'})
team_2017_2018 = team_2017_2018.rename(columns={'TEAMS':'TEAM'})

In [10]:
#Fill NaN spots with 0 under the DK_PTS columns
team_2016_2017[['C_DK_PTS',
       'F_DK_PTS', 'G_DK_PTS', 'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS',
       'SG_DK_PTS']] = team_2016_2017[['C_DK_PTS',
       'F_DK_PTS', 'G_DK_PTS', 'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS',
       'SG_DK_PTS']].fillna(value=0)
team_2017_2018[['C_DK_PTS',
       'F_DK_PTS', 'G_DK_PTS', 'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS',
       'SG_DK_PTS']] = team_2017_2018[['C_DK_PTS',
       'F_DK_PTS', 'G_DK_PTS', 'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS',
       'SG_DK_PTS']].fillna(value=0)

In [11]:
#Combine G and SG columns and Fand PF columns. Drop F and G columns
team_2016_2017['PF_DK_PTS'] = team_2016_2017['PF_DK_PTS'] + team_2016_2017['F_DK_PTS']
team_2016_2017['SG_DK_PTS'] = team_2016_2017['SG_DK_PTS'] + team_2016_2017['G_DK_PTS']
team_2016_2017 = team_2016_2017.drop(['F_DK_PTS','G_DK_PTS'],axis=1)

team_2017_2018['PF_DK_PTS'] = team_2017_2018['PF_DK_PTS'] + team_2017_2018['F_DK_PTS']
team_2017_2018['SG_DK_PTS'] = team_2017_2018['SG_DK_PTS'] + team_2017_2018['G_DK_PTS']
team_2017_2018 = team_2017_2018.drop(['F_DK_PTS','G_DK_PTS'],axis=1)

In [12]:
DFS_2016_2017 = DFS_2016_2017.rename(index=str, columns={'VENUE\nR/H':'ROAD_HOME','USAGE \nRATE (%)':'USAGE','POSITION': 'DK_POS','Unnamed: 9':'FD_POS', 'Unnamed: 10':'YH_POS',
       'SALARY ($)':'DK_SAL', 'Unnamed: 12':'FD_SAL', 'Unnamed: 13':'YH_SAL', 'FANTASY POINTS SCORED':'DK_PTS',
       'Unnamed: 15':'FD_PTS', 'Unnamed: 16':'YH_PTS'})
DFS_2016_2017.drop(['DK_PTS','YH_POS', 'YH_SAL','YH_PTS','FD_POS','FD_SAL','FD_PTS'], axis=1,inplace=True)
DFS_2016_2017 = DFS_2016_2017.reset_index(drop=True)

DFS_2017_2018 = DFS_2017_2018.rename(index=str, columns={'VENUE\nR/H':'ROAD_HOME','USAGE \nRATE (%)':'USAGE','POSITION': 'DK_POS','Unnamed: 9':'FD_POS', 'Unnamed: 10':'YH_POS',
       'SALARY ($)':'DK_SAL', 'Unnamed: 12':'FD_SAL', 'Unnamed: 13':'YH_SAL', 'FANTASY POINTS SCORED':'DK_PTS',
       'Unnamed: 15':'FD_PTS', 'Unnamed: 16':'YH_PTS'})
DFS_2017_2018.drop(['DK_PTS','YH_POS', 'YH_SAL','YH_PTS','FD_POS','FD_SAL','FD_PTS'], axis=1,inplace=True)
DFS_2017_2018 = DFS_2017_2018.reset_index(drop=True)

# Merge DataFrames

In [13]:
#Merge Player DFs with DFS DFs
player_2016_2017 = pd.merge(player_2016_2017,DFS_2016_2017,how='left')
player_2017_2018 = pd.merge(player_2017_2018,DFS_2017_2018,how='left')

In [14]:
player_2016_2017['USAGE'] = (player_2016_2017['USAGE'] / 100) * player_2016_2017['MINUTES']
player_2017_2018['USAGE'] = (player_2017_2018['USAGE'] / 100) * player_2017_2018['MINUTES']

In [15]:
player_2016_2017 = player_2016_2017.drop(['DATASET','FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'OR',
       'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS', 'DT_Double'], axis=1)
player_2016_2017 = player_2016_2017.dropna()

player_2017_2018 = player_2017_2018.drop(['DATASET','FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'OR',
       'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS', 'DT_Double'], axis=1)
player_2017_2018 = player_2017_2018.dropna()



In [16]:
team_2016_2017 = team_2016_2017.drop(['DATASET','F', 'PTS', 'POSS','FG', 'FGA', '3P', '3PA',
                                       'FT', 'FTA', 'OR', 'DR', 'TOT', 'A', 'PF', 'ST', 
                                      'TO', 'TO TO', 'BL','1Q', '2Q', '3Q', '4Q','MIN','OT1', 'OT2', 'OT3', 'OT4', 'MAIN REF', 'CREW', 'BOX SCORE',
                                      'ODDS','MOVEMENTS','HALFTIME', 'MAIN REF', 'CREW','SPREAD',
                                        'OPENING ODDS', 'CLOSING ODDS', 'OPENING SPREAD','MONEYLINE'],axis=1)
team_2017_2018 = team_2017_2018.drop(['DATASET','F', 'PTS', 'POSS','FG', 'FGA', '3P', '3PA',
                                       'FT', 'FTA', 'OR', 'DR', 'TOT', 'A', 'PF', 'ST', 
                                      'TO', 'TO TO', 'BL','1Q', '2Q', '3Q', '4Q','MIN','OT1', 'OT2', 'OT3', 'OT4', 'MAIN REF', 'CREW', 'BOX SCORE', 
                                      'ODDS','MOVEMENTS','HALFTIME', 'MAIN REF', 'CREW','SPREAD',
                                        'OPENING ODDS', 'CLOSING ODDS','OPENING SPREAD', 'MONEYLINE'],axis=1)

team_2016_2017.columns

Index(['DATE', 'TEAM', 'VENUE', 'PACE', 'OEFF', 'DEFF', 'REST DAYS',
       'STARTING LINEUPS', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'OPENING TOTAL', 'FAVORITE', 'SPREAD_10+', 'C_DK_PTS',
       'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS', 'SG_DK_PTS'],
      dtype='object')

# Add Features That Represent the Opponent

In [17]:
#create list of the statistics you want to get the opponent's value for
stats = ['C_DK_PTS', 'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS','SG_DK_PTS']
len(stats)

5

In [18]:
#function that appends a DF of the stat to a list
def add_opponent_stat_column(stat,df):
    frames = [df]
    stat_df = pd.DataFrame(df[[stat]])
    stat_list = []
    for i,row in stat_df.iterrows():
        stat_list.append(row[0])
    opponent_stat_list = []
    
    for i,team in enumerate(stat_list):
        if i % 2 == 0:
            opponent_stat_list.append(stat_list[i+1])
        else:
            opponent_stat_list.append(stat_list[i-1])

    opponent_stat_list_df = pd.DataFrame(opponent_stat_list,columns=['opp_'+stat])

    frames.append(opponent_stat_list_df)
    
    df = pd.concat(frames,axis=1)

    return df

In [19]:
#Append the opponents DK_PTS allowed
for stat in stats:
    team_2016_2017 = add_opponent_stat_column(stat,team_2016_2017)

for stat in stats:
    team_2017_2018 = add_opponent_stat_column(stat,team_2017_2018)

#Rename columns to opponent DK_PTS allowed
team_2016_2017 = team_2016_2017.rename(columns={'opp_C_DK_PTS':'opp_C_allowed',
       'opp_PF_DK_PTS':'opp_PF_allowed', 'opp_PG_DK_PTS':'opp_PG_allowed',
                               'opp_SF_DK_PTS':'opp_SF_allowed','opp_SG_DK_PTS':'opp_SG_allowed'})

team_2017_2018 = team_2017_2018.rename(columns={'opp_C_DK_PTS':'opp_C_allowed',
       'opp_PF_DK_PTS':'opp_PF_allowed', 'opp_PG_DK_PTS':'opp_PG_allowed',
                               'opp_SF_DK_PTS':'opp_SF_allowed','opp_SG_DK_PTS':'opp_SG_allowed'})

# Add Rolling Stats

In [20]:
def join_rolling_stat(input_df,teams,stat,rolls = [5]):
    result = []
    mini_df = input_df[['TEAM',stat]]

    for team in teams:
        df = pd.DataFrame(mini_df.loc[lambda df: input_df.TEAM == team])
        for roll in rolls:
            df[stat + '_roll' + str(roll)] = df[stat].rolling(roll, min_periods=1).mean().shift()
        result.append(df)
    result = pd.concat(result)
    result = result.sort_index()
    result = result.drop(['TEAM',stat],axis=1)
    input_df = input_df.join(result)

    return input_df

In [21]:
pace = ['PACE', 'OEFF', 'DEFF']
DvP_stats = ['opp_C_allowed', 'opp_PF_allowed', 'opp_PG_allowed', 'opp_SF_allowed', 'opp_SG_allowed']

In [22]:
for stat in pace:
    team_2016_2017 = join_rolling_stat(team_2016_2017,teams,stat,rolls=[3,5,10,25])
for stat in pace:
    team_2017_2018 = join_rolling_stat(team_2017_2018,teams,stat,rolls=[3,5,10,25])

for stat in DvP_stats:
    team_2016_2017 = join_rolling_stat(team_2016_2017,teams,stat,rolls=[3,5,10,25])
for stat in DvP_stats:
    team_2017_2018 = join_rolling_stat(team_2017_2018,teams,stat,rolls=[3,5,10,25])

In [23]:
opp_stats = ['PACE_roll3', 'PACE_roll5', 'PACE_roll10',
       'PACE_roll25', 'OEFF_roll3', 'OEFF_roll5', 'OEFF_roll10', 'OEFF_roll25',
       'DEFF_roll3', 'DEFF_roll5', 'DEFF_roll10', 'DEFF_roll25']

In [24]:
for stat in opp_stats:
    team_2016_2017 = add_opponent_stat_column(stat,team_2016_2017)

for stat in opp_stats:
    team_2017_2018 = add_opponent_stat_column(stat,team_2017_2018)    

In [25]:
player_stats = ['MINUTES','DraftKings_PTS', 'USAGE']

In [26]:
def join_rolling_stat_player(input_df,players,stat,rolls = [5]):
    result = []
    mini_df = input_df[['PLAYER',stat]]

    for player in players:
        df = pd.DataFrame(mini_df.loc[lambda df: input_df.PLAYER == player])
        for roll in rolls:
            df[stat + '_roll' + str(roll)] = df[stat].rolling(roll, min_periods=1).mean().shift()
        result.append(df)
    result = pd.concat(result)
    result = result.sort_index()
    result = result.drop(['PLAYER',stat],axis=1)
    input_df = input_df.join(result)

    return input_df

In [27]:
unique_players_2016_2017 = player_2016_2017.PLAYER.unique()
unique_players_2017_2018 = player_2017_2018.PLAYER.unique()



In [28]:
for stat in player_stats:
    player_2016_2017 = join_rolling_stat_player(player_2016_2017,unique_players_2016_2017,stat,rolls=[3,5,10,25])
    player_2017_2018 = join_rolling_stat_player(player_2017_2018,unique_players_2017_2018,stat,rolls=[3,5,10,25])

# Merge and Concatenate DataFrames

In [29]:
player_2016_2017 = pd.merge(player_2016_2017,team_2016_2017,on=['DATE','TEAM'])

player_2017_2018 = pd.merge(player_2017_2018,team_2017_2018,on=['DATE','TEAM'])


In [30]:
print(player_2016_2017.shape,player_2017_2018.shape)

(25890, 90) (25356, 90)


In [31]:
frames = [player_2016_2017,player_2017_2018]

In [32]:
player_2016_2018 = pd.concat(frames)

In [33]:
player_2016_2018 = player_2016_2018.reset_index(drop=True)

In [34]:
player_2016_2018.iloc[100:]

,DATE,PLAYER,POSITION,TEAM,OPPONENT,ROAD_HOME,MINUTES,DraftKings_PTS,USAGE,DK_POS,...,opp_PACE_roll10,opp_PACE_roll25,opp_OEFF_roll3,opp_OEFF_roll5,opp_OEFF_roll10,opp_OEFF_roll25,opp_DEFF_roll3,opp_DEFF_roll5,opp_DEFF_roll10,opp_DEFF_roll25
100,10/26/2016,Dorian Finney-Smith,PF,Dallas,Indiana,R,0.45,1.25,0.000000,PF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,10/26/2016,Paul George,SF,Indiana,Dallas,H,42.77,45.00,8.345918,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,10/26/2016,Thaddeus Young,PF,Indiana,Dallas,H,34.22,24.75,5.130687,PF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,10/26/2016,Myles Turner,C,Indiana,Dallas,H,37.48,64.50,9.919329,PF/C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,10/26/2016,Monta Ellis,SG,Indiana,Dallas,H,44.02,43.25,6.738303,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,10/26/2016,Jeff Teague,PG,Indiana,Dallas,H,36.75,39.00,10.141659,PG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,10/26/2016,Rodney Stuckey,PG,Indiana,Dallas,H,21.90,12.75,3.984834,PG/SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,10/26/2016,Lavoy Allen,PF,Indiana,Dallas,H,10.73,13.25,1.282672,PF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,10/26/2016,Al Jefferson,C,Indiana,Dallas,H,10.90,16.00,3.181026,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,10/26/2016,Joe Young,PG,Indiana,Dallas,H,3.45,0.00,0.000000,SG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create More Features

In [35]:
#creates colums that is a string of all the starters names
player_2016_2018['Starters'] = player_2016_2018[player_2016_2018.columns[27:32]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
#Creates a column 0 or 1 that signifies if the player was a starter for that game
player_2016_2018['Start'] = pd.Series()
for i,x in enumerate(player_2016_2018['PLAYER']):
    player_2016_2018['Start'][i] = player_2016_2018['PLAYER'][i] in player_2016_2018['Starters'][i]

/Users/willhall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
player_2016_2018.columns

Index(['DATE', 'PLAYER', 'POSITION', 'TEAM', 'OPPONENT', 'ROAD_HOME',
       'MINUTES', 'DraftKings_PTS', 'USAGE', 'DK_POS', 'DK_SAL',
       'MINUTES_roll3', 'MINUTES_roll5', 'MINUTES_roll10', 'MINUTES_roll25',
       'DraftKings_PTS_roll3', 'DraftKings_PTS_roll5', 'DraftKings_PTS_roll10',
       'DraftKings_PTS_roll25', 'USAGE_roll3', 'USAGE_roll5', 'USAGE_roll10',
       'USAGE_roll25', 'VENUE', 'PACE', 'OEFF', 'DEFF', 'REST DAYS',
       'STARTING LINEUPS', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'OPENING TOTAL', 'FAVORITE', 'SPREAD_10+', 'C_DK_PTS',
       'PF_DK_PTS', 'PG_DK_PTS', 'SF_DK_PTS', 'SG_DK_PTS', 'opp_C_allowed',
       'opp_PF_allowed', 'opp_PG_allowed', 'opp_SF_allowed', 'opp_SG_allowed',
       'PACE_roll3', 'PACE_roll5', 'PACE_roll10', 'PACE_roll25', 'OEFF_roll3',
       'OEFF_roll5', 'OEFF_roll10', 'OEFF_roll25', 'DEFF_roll3', 'DEFF_roll5',
       'DEFF_roll10', 'DEFF_roll25', 'opp_C_allowed_roll3',
       'opp_C_allowed_roll5', 'opp_C_al

In [37]:
player_2016_2018['DK_PTS_PER_MIN'] = player_2016_2018['DraftKings_PTS'] / player_2016_2018['MINUTES']

In [38]:
player_2016_2018['VALUE'] = player_2016_2018['DraftKings_PTS'] / player_2016_2018['DK_SAL']

In [39]:
player_2016_2018 = player_2016_2018.drop(['STARTING LINEUPS', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'
                                          ,'Unnamed: 39','Starters','VENUE'],axis=1)

In [ ]:
# player_2016_2018 = player_2016_2018.drop(['VENUE','DK_PTS_PER_$$$'], axis=1)

In [ ]:
player_2016_2018.iloc[:,80:]


In [40]:
player_2016_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51246 entries, 0 to 51245
Data columns (total 87 columns):
DATE                     51246 non-null object
PLAYER                   51246 non-null object
POSITION                 51246 non-null object
TEAM                     51246 non-null object
OPPONENT                 51246 non-null object
ROAD_HOME                51246 non-null object
MINUTES                  51246 non-null float64
DraftKings_PTS           51246 non-null float64
USAGE                    51246 non-null float64
DK_POS                   51246 non-null object
DK_SAL                   51246 non-null object
MINUTES_roll3            50226 non-null float64
MINUTES_roll5            50226 non-null float64
MINUTES_roll10           50226 non-null float64
MINUTES_roll25           50226 non-null float64
DraftKings_PTS_roll3     50226 non-null float64
DraftKings_PTS_roll5     50226 non-null float64
DraftKings_PTS_roll10    50226 non-null float64
DraftKings_PTS_roll25    50226 non-

# Save DataFrame as a Pickle

In [41]:
player_2016_2018.to_pickle('player_2016_2018.pkl')

In [ ]:
#This block removes all rows for a player who played less than 10 games in the season.
list1 =[]
for val in player_2016_2017.PLAYER.unique():
    if player_2016_2017.PLAYER.value_counts()[val] < 10:
        list1.append(val)
        
for player in list1:
    player_2016_2017 = player_2016_2017[player_2016_2017.PLAYER != player]

player_2016_2017.info()